In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 582.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.2 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import nltk
import string
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
# Load datasets
def load_data(file_path):
    df = pd.read_csv(file_path, header=None, names=["data"])
    df[["tweet", "label"]] = df["data"].str.split(";", expand=True)
    df.drop(columns=["data"], inplace=True)
    return df

In [6]:
train_df = load_data("/content/drive/MyDrive/train.txt")
val_df = load_data("/content/drive/MyDrive/val.txt")
test_df = load_data("/content/drive/MyDrive/test.txt")

In [7]:
# Handle missing values
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [8]:
# Text preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [9]:
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Remove stopwords & lemmatize
    return " ".join(words)

In [10]:
train_df["tweet"] = train_df["tweet"].apply(preprocess_text)
val_df["tweet"] = val_df["tweet"].apply(preprocess_text)
test_df["tweet"] = test_df["tweet"].apply(preprocess_text)

In [11]:
# Encode labels
label_encoder = LabelEncoder()
train_df["label"] = label_encoder.fit_transform(train_df["label"])
val_df["label"] = label_encoder.transform(val_df["label"])
test_df["label"] = label_encoder.transform(test_df["label"])

In [12]:
# Save label encoder classes
with open("label_encoder.json", "w") as f:
    json.dump(label_encoder.classes_.tolist(), f)

In [13]:
# Tokenization
max_words = 10000  # Max vocabulary size
max_len = 128  # Max sequence length
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["tweet"])

In [14]:
# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_df["tweet"])
val_sequences = tokenizer.texts_to_sequences(val_df["tweet"])
test_sequences = tokenizer.texts_to_sequences(test_df["tweet"])

In [15]:
# Padding sequences
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_len, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding="post", truncating="post")

In [16]:
# Convert labels to categorical (one-hot encoding)
num_classes = len(label_encoder.classes_)
train_labels = to_categorical(train_df["label"], num_classes=num_classes)
val_labels = to_categorical(val_df["label"], num_classes=num_classes)
test_labels = to_categorical(test_df["label"], num_classes=num_classes)

In [17]:
# Build BiGRU model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, dropout=0.3, recurrent_dropout=0.3)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dropout(0.3),  # Prevents overfitting
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
# Compile model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Implement early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

In [20]:
# Train model
epochs = 20
batch_size = 64
history = model.fit(
    train_padded, train_labels,
    validation_data=(val_padded, val_labels),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping]
)

Epoch 1/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 144s 539ms/step - accuracy: 0.3773 - loss: 1.5175 - val_accuracy: 0.8140 - val_loss: 0.5374
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 135s 540ms/step - accuracy: 0.8667 - loss: 0.4142 - val_accuracy: 0.9110 - val_loss: 0.2684
Epoch 3/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 133s 531ms/step - accuracy: 0.9309 - loss: 0.2073 - val_accuracy: 0.9155 - val_loss: 0.2340
Epoch 4/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 131s 522ms/step - accuracy: 0.9546 - loss: 0.1362 - val_accuracy: 0.9260 - val_loss: 0.2406
Epoch 5/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 130s 520ms/step - accuracy: 0.9649 - loss: 0.1008 - val_accuracy: 0.9250 - val_loss: 0.2304
Epoch 6/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 131s 523ms/step - accuracy: 0.9728 - loss: 0.0829 - val_accuracy: 0.9260 - val_loss: 0.2502
Epoch 7/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 130s 522ms/step - accuracy: 0.9750 - loss: 0.0783 - val_accuracy: 0.9265 - val_loss: 0.2557
Epoch 8/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 130s 522ms/step - accuracy: 0.9832 -

In [21]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.9269 - loss: 0.2273
Test Accuracy: 0.9275


In [23]:
# Save model and tokenizer
model.save("bigru_emotion_model.keras")  # Recommended Keras format
tokenizer_json = tokenizer.to_json()
with open("tokenizer.json", "w") as f:
    f.write(tokenizer_json)

print("Model and tokenizer saved successfully!")

Model and tokenizer saved successfully!


In [29]:
# Load the trained model and tokenizer for real-time predictions
model = tf.keras.models.load_model("bigru_emotion_model.keras")

In [30]:
# Load tokenizer correctly
from tensorflow.keras.preprocessing.text import tokenizer_from_json

with open("tokenizer.json", "r") as f:
    tokenizer_json = json.load(f)

# Convert dictionary to JSON string before loading
tokenizer = tokenizer_from_json(json.dumps(tokenizer_json))


In [31]:
with open("label_encoder.json", "r") as f:
    label_classes = json.load(f)

label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(label_classes)


In [32]:
# Function to predict emotion from user input
def predict_emotion(text):
    processed_text = preprocess_text(text)  # Preprocess input text
    sequence = tokenizer.texts_to_sequences([processed_text])  # Convert to sequence
    padded_sequence = pad_sequences(sequence, maxlen=128, padding="post", truncating="post")  # Pad sequence

    prediction = model.predict(padded_sequence)  # Get model prediction
    predicted_label = np.argmax(prediction)  # Get label index
    emotion = label_encoder.inverse_transform([predicted_label])[0]  # Convert index to label

    return emotion


In [35]:
# Take user input and predict emotion
while True:
    user_input = input("Enter a sentence (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        print("Exiting...")
        break
    emotion = predict_emotion(user_input)
    print(f"Predicted Emotion: {emotion}")

Enter a sentence (or type 'exit' to quit): I can't stop crying, everything feels so hopeless.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicted Emotion: sadness
Enter a sentence (or type 'exit' to quit): I am so nervous about my exam results tomorrow.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Emotion: fear
Enter a sentence (or type 'exit' to quit): I just received the best news of my life, I am thrilled!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted Emotion: joy
Enter a sentence (or type 'exit' to quit): I feel so loved and appreciated by my friends and family
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Predicted Emotion: love
Enter a sentence (or type 'exit' to quit): How could they betray me like this? I am so mad!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Predicted Emotion: anger
Enter a sentence (or type 'exit' to quit): She planned a surprise party for my birthday, and I was shocked!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Predicted Emotion: surprise
Enter a sentence (or type 'exit' to 